# LSTM with Attention Keras
Rob Trying to make a LSTM with attention

In [2]:
import tensorflow
import keras
import numpy as np
import pandas as pd

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

## Data Preprocessing

In [11]:
df = pd.read_csv('../../../Comments_FanofGame.csv')

In [34]:
labels = df['fan_of_team_playing'].values
docs = df['comment_body'].values

vocab_size = 100000
encoded_docs = [one_hot(str(d), vocab_size) for d in docs]

In [39]:
# pad documents to a max length of 500 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

## Create Model

In [43]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=3, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 8)            800000    
_________________________________________________________________
flatten_6 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 801       
Total params: 800,801
Trainable params: 800,801
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
2018028/2018028 [==============================] - 426s 211us/step - loss: 0.6444 - acc: 0.6172
Epoch 2/3
2018028/2018028 [==============================] - 427s 211us/step - loss: 0.6336 - acc: 0.6331
Epoch 3/3
2018028/2018028 [==============================] - 426s 211us/step - loss: 0.6267 - acc: 0.6417
Accuracy: 65.386506


# Glove embeddings

In [58]:
## Use pretraineed glove embeddings

docs_string =[str(doc) for doc in docs]
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs_string)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs_string)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[[   4  146  404 ...    0    0    0]
 [   4   14   83 ...    0    0    0]
 [2134   28    6 ...    0    0    0]
 ...
 [   8   66  192 ...    0    0    0]
 [ 250  416  250 ...    0    0    0]
 [  27    2  166 ...    0    0    0]]


In [59]:
f = open('/home/robmulla/Documents/W266/glove/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [61]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=3, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 100)          16079400  
_________________________________________________________________
flatten_10 (Flatten)         (None, 10000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 10001     
Total params: 16,089,401
Trainable params: 10,001
Non-trainable params: 16,079,400
_________________________________________________________________
None
Epoch 1/3
2018028/2018028 [==============================] - 59s 29us/step - loss: 0.6936 - acc: 0.5601
Epoch 2/3
2018028/2018028 [==============================] - 58s 29us/step - loss: 0.6927 - acc: 0.5615
Epoch 3/3
2018028/2018028 [==============================] - 34s 17us/step
Accuracy: 56.432369


# Predict?

In [143]:
predict_prob = model.predict(padded_docs)
predictions = model.predict(padded_docs)

In [135]:
df = pd.DataFrame(index=docs_string)

In [148]:
df['predicted_class'] = predictions
df['actual_class'] = labels
df['predic_prob'] = predict_prob
df['prob_off'] = abs(df['predicted_class'] - df['predic_prob'])

In [153]:
df.sort_values('prob_off').head()

,predicted_class,actual_class,predic_prob,prob_off
"Pretty sure that was correct as well, nothing noting that the time run off would occur\n\n>ARTICLE 2. PERIOD EXTENDED. At the election of the opponent, a period may be extended for one untimed down, if any of\nthe following occurs during a down during which time in the period expires:\n(a) If there is a live-ball foul by the defensive team that is accepted, the offensive team may choose to extend the period by\nan untimed down after enforcement of the penalty. If the first or third period is not so extended, any accepted penalty is\nenforced before the start of the succeeding period.\n(b) If there is a foul by the offense, there shall be no extension of the period. If the foul occurs on the last play of the half, a\nscore by the offense is not counted. However, the period may be extended for an untimed down, upon the request of the\ndefense, if the offensive team’s foul is for:\n(1) illegal touching of a kick;\nNote: The period may also be extended for a “first touching” violation.\n(2) fair-catch interference;\n(3) a palpably unfair act;\n(4) a personal foul or unsportsmanlike conduct foul committed prior to a safety, an interception of a forward pass, the\nrecovery of a backward pass or fumble, or the offensive team failing to reach the line to gain on fourth down; or\n(5) a foul by the kicking team prior to a player of the receiving team securing possession of the ball during a down in\nwhich there is a safety kick, a scrimmage kick, or a free kick.\n(c) If a touchdown is made on the last play of a period, the Try attempt shall be made (except during a sudden-death period).\n(d) If no fair-catch signal is given and the kickers interfere with the receiver’s opportunity to catch a kick, the receiving team\nmay extend the period by a down from scrimmage.\n(e) If a fair-catch is signaled and made, the receivers may choose to extend the period by a fair-catch kick down (10-2-4-a). If\nthe first or third period is not so extended, the receivers may start the succeeding period by either a down from scrimmage\nor fair-catch kick (11-4-3).\n(f) If a fair catch is signaled and the kickers interfere with a receiver’s opportunity to catch a kick, the receiving team may\nextend the period by either a down from scrimmage or a fair-catch kick (10-2-4-a).\n(g) If a safety results from a foul during the last play of a half, the score counts. A safety kick is made if requested by the\nreceivers.\n16 Rule 4, Section 7\n(h) If a double foul occurs during the last down of either half, the period shall be extended by an untimed down.\nExceptions: The half is not extended if:\n(1) both fouls are dead-ball fouls\n(2) there is a major-minor double foul (“5 vs. 15”), and the major foul is by the offense, or if the major foul is a dead ball\nfoul by the defense (see 14-5-1-Item 1.)\nNote: Dead ball, personal, unsportsmanlike conduct, or taunting fouls by either team at the end of a half are enforced\non the ensuing kickoff (14-4-9-Exc. 1).\n(3) there is a double foul with a change of possession (“clean hands” rule, see 14-5-2) that does not involve a replay of\nthe down. If a double foul occurs on the last play of the first or third periods, the period is not extended.\nIf the first or third period is extended for any reason, or if a touchdown occurs during the last play of such a period, any\nadditional play, including a Try attempt, shall be completed before the teams change goals.\nIf any period is extended for any reason, it shall continue until the completion of a down free from any foul specified in (a)\nthrough (h) above.\n\n",1,True,1.000000e+00,0.000000e+00
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nb

In [155]:
df.sort_values('prob_off').tail(50)

,predicted_class,actual_class,predic_prob,prob_off
"Man, really hoping for the best. You never want to see that.",0,False,0.499996,0.499996
"LMAO that Eagles fan ""this is our town now""",1,True,0.500004,0.499996
Hasnt Crowder done this a few times on fielding kicks?,1,False,0.500004,0.499996
They'll score on 3rd and 6 anyways,0,False,0.499996,0.499996
No responsible Jags fan is cocky at this point.,1,True,0.500004,0.499996
Punching people wearing helmets is the stupidest thing,0,True,0.499996,0.499996
That fan was the genderbent version of Jason Garett,1,False,0.500004,0.499996
someone gonna die climbing those fucking poles,1,False,0.500004,0.499996
That should have been Mahomes third TD. The fuck is this Bray shit?,0,True,0.499997,0.499997
Aren't most Dallas Cowboy fans douche nozzles?,1,False,0.500003,0.499997


In [142]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true=df['actual_class'],
                 y_pred=df['predicted_class'])

array([[756944, 347211],
       [531996, 381877]])

# Train LSTM

In [8]:
df = pd.read_csv('../../../Comments_FanofGame.csv')

labels = df['fan_of_team_playing'].values
docs = df['comment_body'].values

## Use pretraineed glove embeddings

docs_string =[str(doc) for doc in docs]
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs_string)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs_string)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()

f = open('/home/robmulla/Documents/W266/glove/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
#model.add(Flatten())
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

# checkpoint for periodically saving model
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# fit the model
model.fit(padded_docs, labels, epochs=8, verbose=1, validation_split=0.33, callbacks=callbacks_list)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[[   4  146  404 ...    0    0    0]
 [   4   14   83 ...    0    0    0]
 [2134   28    6 ...    0    0    0]
 ...
 [   8   66  192 ...    0    0    0]
 [ 250  416  250 ...    0    0    0]
 [  27    2  166 ...    0    0    0]]
Loaded 1193514 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          16079400  
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 16,159,901
Trainable params: 80,501
Non-trainable params: 16,079,400
_________________________________________________________________
None
Train on 1352078 samples, validate on 665950 samples
Epoch 1/8
1352078/1352078 [==============================] - 2003s 1ms/ste